# Sonderübung 2

## Imports

In [415]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import scipy
from gurobipy import Model, GRB, Var

### Achtung: 
die untenstehenden Befehle funktionieren nur mit einer aktuellen pandas Version: falls beim pandas.drop() Befehl, ein Fehler auftritt , dann schauen, dass pandas geupdated wird zb über "pip install pandas --upgrade" (Version 1.4.2. funktioniert), danach das Notebook komplett neustarten und nicht nur die Importzelle neu ausführen

## Parameter für das Optimierungsproblem

In [416]:
shares = ['AAPL', 'MSFT', 'XOM', 'JNJ', 'GE', 'BRK-B', 'FB', 
          'AMZN', 'WFC','T','GOOGL','PG','GOOG','VZ','PFE',
          'CVX','KO','HD','DIS','BAC','MRK','V','PM','CMCSA',
          'INTC','PEP','CSCO','C','GILD','IBM']

In [417]:
historical_prices = pd.read_excel('historical_prices.xlsx', index_col=0)
historical_returns = historical_prices / historical_prices.shift(1) - 1
historical_returns = historical_returns.drop(['2015-05-13'])
mu = historical_returns.mean()*252 # Estimation of mu
sigma = historical_returns.cov()*252 # Estimation of covariance-matrix

In [418]:
sigma.iloc[0:8,0:8]

,AAPL,MSFT,XOM,JNJ,GE,BRK-B,FB,AMZN
AAPL,0.077915,0.045575,0.031541,0.020971,0.028955,0.027391,0.040927,0.040994
MSFT,0.045575,0.072776,0.027717,0.025104,0.032222,0.029367,0.048484,0.051320
XOM,0.031541,0.027717,0.059183,0.022321,0.028922,0.027715,0.024145,0.024970
JNJ,0.020971,0.025104,0.022321,0.026636,0.019970,0.021105,0.022804,0.023365
GE,0.028955,0.032222,0.028922,0.019970,0.043824,0.026067,0.029822,0.028163
BRK-B,0.027391,0.029367,0.027715,0.021105,0.026067,0.031443,0.026365,0.025164
FB,0.040927,0.048484,0.024145,0.022804,0.029822,0.026365,0.106992,0.075159
AMZN,0.040994,0.051320,0.024970,0.023365,0.028163,0.025164,0.075159,0.121713


In [419]:
Sigma = sigma.values
Mu = mu.values
lamb = 10
n = Mu.size

In [420]:
Mu_shape = Mu.reshape(n,1)

## Aufgabe 2 b) 
Zeigen Sie, dass die Zielfunktion von $P$ mit den gegebenen Daten $\Sigma, \mu$ und $\lambda$ gleichmäßig konvex ist.

In [421]:
eigenvalues, _ = np.linalg.eig(lamb*Sigma)
print(min(eigenvalues))

0.007788373017431655


## 2 c) Frank Wolfe
Implementieren Sie das Verfahren aus dem Algorithmus auf dem Aufgabenblatt. Geben Sie am Ende aus, wie viel in welche Aktien investiert wird. Beachten Sie die Hinweise auf dem Aufgabenblatt.

In [422]:
eps = 1e-4

In [423]:
def problem_S(x_k,d_k):
    m = Model("S")
    
    t = m.addVar(vtype = GRB.CONTINUOUS, lb=0, ub=1, name = "t")
    
    print(f"x_k{len(x_k)}, d_k:{d_k.shape}, Mu:{Mu.shape}, sigma:{Sigma.shape}")
    
    
    lin = m.addMVar(n, vtype = GRB.CONTINUOUS, lb=float('-inf'), ub=float('inf'), name = "t")
    for i in range(n):
        m.addConstr(x_k[i]+t*d_k[i] == lin[i])
    
    
    
    
    m.setObjective(-Mu@lin+1/2*lamb*lin@Sigma@lin)
    
    m.optimize()
    return m.x, m.ObjVal

In [424]:
def problem_Q(xk):
    m = Model("Q")
    
    x = m.addMVar(len(shares), lb=0, name="x")
    m.addConstr(x.sum() == 1)
    
    vec = -Mu+lamb*Sigma@xk
    m.setObjective(sum((x[i]-xk[i])*vec[i] for i in range(len(vec))))
    
    m.optimize()
    return m.x, m.ObjVal

In [425]:
k = 0
x_k = np.repeat(1/30, 30)
y_k, v_k = problem_Q(x_k)

while v_k < -eps:
    d_k = y_k-x_k
    t_k, _ = problem_S(x_k=x_k, d_k=d_k)
    x_k = x_k + t_k*d_k
    k += 1
    y_k, v_k = problem_Q(x_k)

x_quer = x_k
    

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 1 rows, 30 columns and 30 nonzeros
Model fingerprint: 0x70169ea4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-04, 6e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 30 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.2716359e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective -4.271635934e-01
x_k30, d_k:(30,), Mu:(30,), sigma:(30, 30)


GurobiError: Cannot multiply with an MLinExpr from the left

## 2 d) Python Lösung
Lösen Sie $P$ direkt mit Python und vergleichen Sie ihren Optimalpunkt mit dem aus Teil c). Ist dieses Ergebnis zu erwarten? Verwenden Sie Aufgabenteil b).

In [ ]:
m = Model("d)")

x = m.addMVar(len(shares), lb=0)

m.addConstr(x.sum() == 1)

m.setObjective(-Mu@x+1/2*lamb+x@Sigma@x)

m.optimize()

m.x

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 1 rows, 30 columns and 30 nonzeros
Model fingerprint: 0x237ad255
Model has 465 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-03, 6e-01]
  QObjective range [4e-02, 4e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 1 rows, 30 columns, 30 nonzeros
Presolved model has 465 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 29
 AA' NZ     : 4.350e+02
 Factor NZ  : 4.650e+02
 Factor Ops : 9.455e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.26656382e+05 -3.33219049e+05  2.40e+04 1.16e+00  9.99e+05     0s
   1   7.71006790e+04 -7.77322579e+04  1.30e+03 2.0

[3.3324723876499305e-14,
 7.8187436823026e-14,
 7.968761214913958e-14,
 1.171366613700064e-13,
 1.0030272622713319e-13,
 6.619579634962677e-14,
 5.161985384674959e-09,
 0.999999994834639,
 4.710561254394728e-14,
 1.5048107950348932e-13,
 5.736426705355587e-13,
 7.991574077707137e-14,
 5.762538348612832e-13,
 7.966783966105839e-14,
 6.593070931385933e-14,
 6.047093563178207e-14,
 1.1798014682196639e-13,
 1.5816027505528656e-13,
 5.428072785310513e-14,
 4.516185548116333e-14,
 5.338891742188804e-14,
 1.0605896253366183e-13,
 1.9446014904307406e-13,
 1.0539422755556787e-13,
 5.325983967596729e-14,
 1.0389805683888179e-13,
 4.9687437352778106e-14,
 4.06732566125916e-14,
 3.915987104545097e-14,
 4.703248770381669e-14]

### Vergleich